# L1: Create Agents to Research and Prepare a Quiz

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

# Install dependencies for Google Colab

In [1]:
!pip install crewai crewai_tools langchain_openai langchain-community langchain-groq

# Import Requirements

* Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew, LLM
from langchain_openai.chat_models import ChatOpenAI


# crewAI Tools
* [Serper](https://serpapi.com/)

## Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

In [3]:
from crewai_tools import SerperDevTool

# Setup Groq and Serper API Keys

## Setting Up Groq API Key

Groq provides access to AI models via its API. Follow these steps to get your API key:

### Steps:
1. Visit [Groq API Dashboard](https://console.groq.com/).

2. Sign in or Sign up

3. Generate API Key
   - Navigate to the **API Keys** section.
   - Click **Create New Key**.
   - Copy and securely store the generated key.


## Setting Up Serper API Key
Serper is a search API that allows programmatic access to Google search results.

### Steps:
1. Visit [Serper API Portal](https://serpapi.com/)
2. Sign in or Sign up
3. Register if you are a new user. Otherwise, log in with your credentials.
4. Generate API Key
   - Click Generate API Key.
   - Copy and securely store the API key.

In [4]:
from google.colab import userdata
import os

os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
os.environ["SERP_API_KEY"] = userdata.get("SERP_API_KEY")

In [5]:
# llm = ChatOpenAI(
#     model="llama-3.3-70b-versatile",
#     api_key=GROQ_API_KEY,
#     base_url="https://api.groq.com/openai/v1",
#     temperature=0.2
# )

llm = LLM(model="groq/llama-3.3-70b-versatile")

# Create Agents

In [6]:
# Initialize tools
search_tool = SerperDevTool()

# Start Working on your Tool

In [7]:
# Agent 1: Task Organizer
daily_task_agent = Agent(
    role="Task Management Assistant",
    goal="Helps users organize their daily tasks by understanding their routine, setting priorities, tracking incomplete tasks, and ensuring deadlines are met.",
    backstory=(
        "An AI assistant designed to help users manage their time effectively. "
        "It reads the user's daily schedule, suggests improvements, and ensures high-priority tasks are completed on time. "
        "Tracks unfinished tasks and carries them forward, while allowing users to assign priority levels (High, Medium, Low)."
    ),
    verbose=True,
    llm=llm
)

# Agent 2: Self-Care Coach
self_care_agent = Agent(
    role="Self-Care and Wellness Coach",
    goal="Encourages self-care habits by providing reminders, wellness tips, stress management techniques, and habit tracking tailored to the user’s needs.",
    backstory=(
        "A supportive and compassionate AI coach that promotes well-being. "
        "It listens to users, provides gentle nudges for self-care, and adapts its recommendations based on their mood and routine. "
        "Includes habit tracking for activities such as meditation, hydration, and exercise. "
        "Suggests personalized strategies based on user mood, such as deep breathing exercises when stressed."
    ),
    verbose=True,
    llm=llm
)

# Agent 3: Daily Planner
daily_planner_agent = Agent(
    role="Daily Planner and Routine Guide",
    goal="Helps users create structured and adaptable daily routines that balance productivity and self-care, with dynamic adjustments for unexpected events.",
    backstory=(
        "An AI assistant that assists users in planning their day efficiently. "
        "It organizes schedules, integrates self-care activities, and adjusts plans dynamically to ensure a well-balanced lifestyle. "
        "Allows users to update schedules based on unexpected events and includes a focus timer with break reminders to enhance productivity."
    ),
    verbose=True,
    llm=llm
)

# Define Tasks

In [8]:
# Task 1: Daily Task Management
task_management_task = Task(
    description=(
        "Gather and organize the user's daily tasks, prioritizing them "
        "based on urgency and importance. Ensure unfinished tasks are tracked "
        "and carried over when necessary."
    ),
    expected_output=(
        "Structured daily task list with:\n"
        "- Task name\n- Priority level (High/Medium/Low)\n- Due time (if applicable)\n"
        "- Status (Pending/Completed)\n- Carry-over flag for unfinished tasks"
    ),
    agent=daily_task_agent
)

# Task 2: Self-Care Recommendations
self_care_task = Task(
    description=(
        "Analyze the user's schedule and recommend self-care habits, "
        "such as hydration reminders, breathing exercises, and wellness breaks. "
        "Provide suggestions based on stress levels and energy levels."
    ),
    expected_output=(
        "Personalized self-care plan including:\n"
        "- Suggested self-care activities\n- Recommended time slots\n"
        "- Justification based on user's routine and stress levels"
    ),
    agent=self_care_agent,
    context=[task_management_task]  # Uses task data to avoid interfering with important work
)

# Task 3: Daily Schedule Planning
daily_planning_task = Task(
    description=(
        "Create a structured daily plan for the user, integrating work tasks "
        "with necessary self-care breaks. Adjust dynamically based on real-time updates."
    ),
    expected_output=(
        "Optimized schedule including:\n"
        "- Work tasks with estimated duration\n- Self-care breaks\n- Adjustable time slots"
    ),
    agent=daily_planner_agent,
    context=[task_management_task, self_care_task]  # Uses both tasks to create a balanced plan
)


# Crew

In [9]:

self_care_crew = Crew(
    agents=[daily_task_agent, self_care_agent, daily_planner_agent],
    tasks=[task_management_task, self_care_task, daily_planning_task],
    verbose=True
)

In [13]:
result = self_care_crew.kickoff(inputs = {
    'user_schedule': "Wake up at 7 AM, work from 9 AM to 5 PM, rest at home from 6 PM, sleep at 11pm.",
    'self_care_goals': "Drink more water, meditate, take short breaks, get physically fit.",
    'task_list': "Finish report, call my parents and say hello, fix electric switch."
})

# Agent: Task Management Assistant
## Task: Gather and organize the user's daily tasks, prioritizing them based on urgency and importance. Ensure unfinished tasks are tracked and carried over when necessary.


# Agent: Task Management Assistant
## Final Answer: 
To effectively manage the user's daily tasks, I have compiled a structured list that takes into account the priority levels, due times, and status of each task. Below is the organized task list:

1. Task Name: Morning Exercise
Priority Level: Medium
Due Time: 7:00 AM
Status: Pending
Carry-over Flag: No

2. Task Name: Project Report Submission
Priority Level: High
Due Time: 10:00 AM
Status: Pending
Carry-over Flag: No

3. Task Name: Team Meeting
Priority Level: High
Due Time: 11:00 AM
Status: Pending
Carry-over Flag: No

4. Task Name: Respond to Urgent Emails
Priority Level: High
Due Time: 12:00 PM
Status: Pending
Carry-over Flag: No

5. Task Name: Lunch Break
Priority Level: Low
Due Time: 1:00 PM
Status: Pending
Carry-over Flag